# Linearity in Finance

> To-do list:
* Examining the CAPM model with the efficient frontier and capital market line
* Solving for the security market line using regression
* Examining the APT model and performing a multivariate linear regression
* Understanding linear optimization in portfolio allocation
* Linear optimization using the PuLP package
* Understanding the outcomes of linear programming
* Introduction to integer programming
* Implementing a linear integer programming model with binary conditions
* Solving systems of linear equations with equalities using matrix linear algebra
* Solving systems of linear equations directly with the LU, Cholesky, and QR decomposition
* Solving systems of linear equations indirectly with the Jacobi and Gauss-Seidel method

## CAPM model

$$ R_i = R_f+\beta_i(R_{mkt} - R_f) $$

The `scipy.stats.linregress` function returns the slope of the regression line, the intercept of the regression line, the correlation coefficient, the p-value for a hypothesis test with null hypothesis of a zero slope, and the standard error of the estimate

| Time period | Stock returns | Market returns |
| ----------- | ----------- | ----------- |
| 1 | 0.065 | 0.055 |
|2 |0.0265|-0.09
|3|-0.0593|-0.041
|4|-0.001|0.045
|5|0.0346|0.022






In [1]:
""" Linear regression with SciPy """
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)

print(beta, alpha)

0.5077431878770808 -0.008481900352462384


* The equation describing the SML can be written as:
$$ E(R_i)=R_f +β_i [E(R_M)−R_f] $$

## The Arbitrage Pricing Theory (APT) model

$$ E[R_i]=α_i+β_{i,1}F_1+β_{i,2}F_2+...+β_{i,j}F_j $$

In [2]:
""" Least squares regression with statsmodels """
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8)
                        for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0]  # First column values as Y
x_values = all_values[:, 1:]  # All other values as X
x_values = sm.add_constant(x_values)  # Include the intercept
results = sm.OLS(y_values, x_values).fit()  # Regress and fit the model

In [3]:
results.summary()


/Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     2.481
Date:                Tue, 06 Jun 2023   Prob (F-statistic):              0.454
Time:                        16:05:54   Log-Likelihood:                 13.909
No. Observations:                   9   AIC:                            -11.82
Df Residuals:                       1   BIC:                            -10.24
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1972      0.358     -0.551      0.679      -4.742       4.348
x1            -0.0243      0.421     -0.058      0.963      -5.376       5.327
x2             0.5093      0.276      1.844      0.316      -3.000       4.019
x3             0.6797      0.310      2.193      0.272      -3.258       4.617
x4             0.1343      0.351      0.383      0.767      -4.320       4.589
x5            -0.3118      0.579     -0.538      0.686      -7.669       7.046
x6             0.9525      0.801      1.189      0.445      -9.229      11.134
x7            -0.7214      0.274     -2.636      0.231      -4.199       2.756
==============================================================================
Omnibus:                        1.734   Durbin-Watson:                   2.169
Prob(Omnibus):                  0.420   Jarque-Bera (JB):                0.813
Skew:                           0.262   Prob(JB):                        0.666
Kurtosis:                       1.623   Cond. No.                         37.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""